In [1]:
# discharge.py usage
# by Rui, Aug 2021

In [5]:
import pickle
from offline import discharge
from offline import ReadObs
from offline import ReadOb
from offline import ReadPRD
from offline import ReadQparams
"""
inputs:
    ----------
    reach:
        - reach['area_fits'] contains the parameters of cross-sectional area change  
        - reach['discharge_models'] contains the parameters of 5 discharge models  
        
    reach_stats: 
        - reach_stats['height'], SWOT reach height [m] 
        - reach_stats['width'], SWOT reach width [m]
        - reach_stats['slope'], SWOT reach slope [m/m]
        
outputs:
    ----------
    d_x_area: cross-sectional area change [m^2]
    d_x_area_u: cross-sectional area change uncertainty [m^2]
    metro_q_c : MetroMan discharge gage constrained [m^3/s]
    metro_q_uc : MetroMan discharge gage un-constrained [m^3/s]
    bam_q_c : BAM discharge gage constrained [m^3/s]
    bam_q_uc : BAM discharge gage un-constrained [m^3/s]
    bam_q_c : BAM discharge gage constrained [m^3/s]
    bam_q_uc : BAM discharge gage un-constrained [m^3/s]
    hivdi_q_c : HiVDI discharge gage constrained [m^3/s]
    hivdi_q_uc : HiVDI discharge gage un-constrained [m^3/s]
    momma_q_c : MOMMA discharge gage constrained [m^3/s]
    momma_q_uc : MOMMA discharge gage un-constrained [m^3/s]
    sads_q_c : SADS discharge gage constrained [m^3/s]
    sads_q_uc : SADS discharge gage un-constrained [m^3/s]
    
    **Discharge uncertainties are not implemented at this moment
"""

# load data saved from the RiverObs run on Sac reaches
# note: some of the items under the dict provided in the /data won't be used
obs ='/Users/rui/Desktop/Github/offline-discharge-data-product-\
creation/data/sac_pkl_files/reach25.pkl'
prd ='/Users/rui/Desktop/Github/offline-discharge-data-product-\
creation/data/sac_pkl_files/reach_prd25.pkl'
reach_stats = pickle.load(open(obs, "rb"))
reach = pickle.load(open(prd, "rb"))

In [8]:
# example 1. run discharge module using the .pkl data saved from RiverObs runs
# copied from RiverObs/src/SWOTRiver
discharge_model_values = discharge.compute(
            reach, reach_stats['height'], reach_stats['width'],
            reach_stats['slope'])


# print(discharge_model_values)

TypeError: compute() missing 1 required positional argument: 'reach_d_x_area'

In [9]:
# # example 2. run discharge module using getting the data saved from SWORD/PRD and rivertile
# # read swot rivertile
# rivertile_path = '/Users/rui/Desktop/Github/offline-discharge-data-product-\
# creation/data/rivertile/rivertile.nc'
# obs = ReadObs.Rivertile(rivertile_path)

# # read prd/sword
# sword_path = '/Users/rui/Desktop/Github/offline-discharge-data-product-\
# creation/data/sword/na_apriori_rivers_v07_4q_v8.nc'
# for i in range(len(obs['reach_id'])):
#     prd = ReadPRD.ReachDatabase(sword_path, obs['reach_id'][i])

#     if prd['area_fits']['h_w_nobs'] != -9999:
#         discharge_model_values = discharge.compute(
#                 prd, obs['height'][i], obs['width'][i],
#                 obs['slope'][i])
#         #print(discharge_model_values)

In [11]:
# example 3. run discharge module using getting the data saved from SWORD/PRD and single reach
# read swot reach
rivertile_dir = '/Users/rui/Desktop/Github/offline-discharge-data-product-creation/data/swot/'
reach_id = 77449100061
ob = ReadOb.Rivertile(rivertile_dir)

# read prd/sword
sword_path = '/Users/rui/Desktop/Github/offline-discharge-data-product-\
creation/data/sword/na_apriori_rivers_v07_4q_v8.nc'

prd = ReadPRD.ReachDatabase(sword_path, ob['reach_id'])
    
for i in range(len(ob['height'])):
    
    if prd['area_fits']['h_w_nobs'] != -9999:
        discharge_model_values = discharge.compute(
                prd, ob['height'][i], ob['width'][i],
                ob['slope'][i])
        print(i, prd,'obs',obs['height'][i])

In [3]:
# example 4. extract Q params from Confluence
from pathlib import Path
from offline import discharge
from offline import ReadOb
from offline import ReadQparams
from offline.ReadQparams import extract_alg

r_id = "77449100061"
rivertile_dir = '/Users/rui/Desktop/Github/offline-discharge-data-product-creation/data/swot/'
rivertile_file = Path(rivertile_dir + '/' + f'{r_id}_SWOT.nc')

ob = ReadOb.Rivertile(rivertile_file)


alg_dir = "/Users/rui/Desktop/Github/offline-discharge-data-product-creation/data/flpe/"
reach = extract_alg(alg_dir, r_id)

for i in range(len(ob['height'])):
    
    if reach['discharge_models']['unconstrained']['BAM']['n'] is not None:
        discharge_model_values = discharge.compute(
                reach, ob['height'][i], ob['width'][i],
                ob['slope'][i], ob['d_x_area'][i])
        #print(i, discharge_model_values)
